# Welcome to the AIinAfrica Hackaton!!

## Climate Change: Earth Surface Temperature Data contest
### Introduction




In this notebook we explore a dataset of monthly temperatures across the globe dated from 1750 to 2015. We will investigate if temperatures have been rising and we will compare Machine Learning models to predict accurately future temperatures in countries an cities of choice.     
    
The dataset we use is the [World Bank Climate Change Data from](https://data.worldbank.org/topic/climate-change) and can be downloaded from this [Kaggle page](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data).

It contains 3 csv files:

`GlobalLandTemperaturesByCity.csv`

`GlobalLandTemperaturesByCountry.csv`

`GlobalTemperatures.csv`

In the first part we analyze the dataset to verify some statistical properties, like the stationarity. Stationarity means that the statistical properties of a process generating a time series do not change over time. It does not mean that the series does not change over time, just that the way it changes does not itself change over time. 

Then we check is the data is seasonal, as we expect from land and sea temperatures.

Finally, we use a Natural diasaster data  taken from https://ourworldindata.org/natural-disasters (data published by EMDAT (2019): OFDA/CRED International Disaster Database, Université catholique de Louvain – Brussels – Belgium), that includes the following files:

`number-of-natural-disaster-events.csv`

`natural-disasters-by-type.csv`

`economic-damage-from-natural-disasters.csv`

We will show how there has been an increase in natural disaster occurrences in the last 50 years and how they are correlated to climate change.

#### Importing required modules 

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import datetime as dt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

#### **Loading the datasets**

In [ ]:
import zipfile
with zipfile.ZipFile("data/GlobalTemperatures.csv.zip","r") as zip_ref:
    zip_ref.extractall("data")
with zipfile.ZipFile("data/GlobalLandTemperaturesByCountry.csv.zip","r") as zip_ref:
    zip_ref.extractall("data")    

In [ ]:
global_temperatures = pd.read_csv('data/GlobalTemperatures.csv')
country_temperatures = pd.read_csv('data/GlobalLandTemperaturesByCountry.csv')
SA = pd.read_csv('data/SATemperaturesbyCity.csv')

In [ ]:
#city_temperatures = pd.read_csv('data/GlobalLandTemperaturesByCity.csv')
#SA = city_temperatures[city_temperatures.Country == 'South Africa']
#SA.set_index('dt', inplace=True)
#SA.to_csv('data/SATemperaturesbyCity.csv')

In [ ]:
#SA.to_csv('data/SATemperaturesbyCity.csv')

In [ ]:
SA.shape

In [ ]:
SA.head()

In [ ]:
country_temperatures.head()

In [ ]:
country_temperatures.shape

In [ ]:
global_temperatures.head()

In [ ]:
global_temperatures.shape

In [ ]:
# Convert dt from string to datetime
global_temperatures['dt'] = pd.to_datetime(global_temperatures.dt)
global_temperatures['year'] = global_temperatures['dt'].map(lambda x:x.year)
global_temperatures['month'] = global_temperatures['dt'].map(lambda x:x.month)

country_temperatures['dt'] = pd.to_datetime(country_temperatures.dt)
country_temperatures['year'] = country_temperatures['dt'].map(lambda x:x.year)
country_temperatures['month'] = country_temperatures['dt'].map(lambda x:x.month)

In [ ]:
# Convert dt from string to datetime
SA['dt'] = pd.to_datetime(SA.dt)
SA['year'] = SA['dt'].map(lambda x:x.year)
SA['month'] = SA['dt'].map(lambda x:x.month)

# Is The Global Temperature Really Rising??

In [ ]:
# We take the avg temperature per year
global_mean=global_temperatures.groupby(['year'])['LandAverageTemperature'].mean().reset_index()
global_mean.dropna(inplace=True)

In [ ]:
global_mean.head()

In [ ]:
# Visualize Plot
trace=go.Scatter(
    x=global_mean['year'],
    y=global_mean['LandAverageTemperature'],
    mode='lines',
)
layout = go.Layout(
    title=go.layout.Title(
        text='Global Average Temperatures 1750-2015',
        xref='paper',
        x=0
    ))

data=[trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='line-mode')
#title="Global Average Temperatures 1750-2015"

In climate science, temperature anomaly represents temperature pattern accurately than absolute temperature. It is a measure of the departure from baseline temperature. Basically, it indicates how much warmer or cooler it is than the baseline. The baseline used here is the average temperature over the 30 year period 1951-1980 (base period used by NASA).<br><br>The temperature anomaly values are stored in a new column in the dataframe.

In [ ]:
#global_mean.head()
baseline_temp = global_mean.loc['1951':'1980'].mean()['LandAverageTemperature'] 
baseline_temp

In [ ]:
# Calculate the global baseline temperature
baseline_temp = global_mean.loc['1951':'1980'].mean()['LandAverageTemperature'] 

# Create the temperature anomaly column
global_mean['TemperatureAnomaly'] = global_mean['LandAverageTemperature'] - baseline_temp

global_mean.head()

In [ ]:
# Visualize Plot with Temperature Anomalies starting from 1900, as measurements were more accurate 
trace=go.Scatter(
    x=global_mean[global_mean['year'] > 1900].year,
    y=global_mean[global_mean['year'] > 1900].TemperatureAnomaly,
    mode='lines',
    )


layout = go.Layout(
    title=go.layout.Title(
        text='Global Temperature Anomalies 1900-2015',
        xref='paper',
        x=0
    ))
data=[trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='line-mode')

The plot shows the warming of Earth. The warming is more pronounced from about 1920 onwards. In 2015, the Earth is 0.76 degree Celsius warmer than the normal. The result above confirms the scientific consensus that the Earth is warming.

#### Dicky Fuller Stationarity Test

In [ ]:
# Dicky Fuller Stationarity Test (https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1911068)
from statsmodels.tsa.stattools import adfuller

def adf_test(series, title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4], index=labels)

    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")
        

adf_test(global_mean.LandAverageTemperature, title='DFT')

## Time Series Deconstruction:
#### 1. Trend: Reflects long term progression of series
#### 2. Seasonality: Reflects seasonal variation and checks if seasonal pattern exists in the time series
#### 3. Residual: Shows irregular component (or "noise") at time t, which describes random, irregular influences

In [ ]:
ts = global_temperatures[['dt','LandAverageTemperature']]
rng = pd.date_range(start='1901-01-01', end='2015-12-01', freq = 'MS')

data = ts[ts.dt.dt.year>1900].LandAverageTemperature.values
series = pd.Series(data=data, index=rng)
len(data), len(series)


In [ ]:
## ETS Graph (Error Trend Seasonality)

from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(series, model='additive')
plt.figure(figsize=(15,8))
result.plot();

In [ ]:
# There exists 12 months seasonality in the dataset
result.seasonal.loc['2010-01-01':'2012-01-01'].plot()

## Now we can look at measurements in January and July

In [ ]:
dict_month = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
#global_temperatures['month']=global_temperatures['month'].map(dict_month)

In [ ]:
def plot_month(month1,month2):
    a = global_temperatures[global_temperatures['month'] == month1]
    b = global_temperatures[global_temperatures['month'] == month2]
    trace0 = go.Scatter(
    x = a['year'],
    y = a['LandAverageTemperature'],
    mode = 'lines',
    name = dict_month[month1]
    )
    
    trace1 = go.Scatter(
    x = b['year'],
    y = b['LandAverageTemperature'],
    mode = 'lines',
    name = dict_month[month2]
    )
    layout = go.Layout(
    title=go.layout.Title(
        text="Global Average Temperatures in {} and {} between 1900-2015".format(dict_month[month1], dict_month[month2]),
        xref='paper',
        x=0
    ))
    data = [trace0,trace1]
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='line-mode')
    
plot_month(1,7)


Looks like July temperatures have always been higher. What if you split the dataset between the northern and southern emisphere and draw this graph on the two datasets?

## Interactive Map

In [ ]:
# We remove the duplicated countries and the continents (in the analysis, we don't consider the presence of 
# colonies at this the countries) and countries for which no information about the temperature

country_temp = country_temperatures[~country_temperatures['Country'].isin(
    ['Denmark', 'France', 'Europe', 'Netherlands','United Kingdom', 'Africa', 'Europe', 'Asia', 'South America', 'North America'])]

country_temp = country_temp.replace(
   ['Denmark (Europe)', 'France (Europe)', 'Netherlands (Europe)', 'United Kingdom (Europe)'],
   ['Denmark', 'France', 'Netherlands', 'United Kingdom'])

#Let's average temperature for each country
countries = np.unique(country_temp['Country'])
mean_temp = []
for country in countries:
    mean_temp.append(country_temp[country_temp['Country'] == country]['AverageTemperature'].mean())

In [ ]:
data = [ dict(
        type = 'choropleth',
        #autocolorscale = True,
        #colorscale = 'RdYlGn',
        #reversescale = True,
        showscale = True,
        locations = countries,
        z = mean_temp,
        locationmode = 'country names', 
        marker = dict(
            line = dict(color = 'rgb(200,200,200)', width = 0.5)),
            colorbar = dict(autotick = True, tickprefix = '', 
            title = 'Temperature')
            )
       ]

layout = dict(
    title = 'Average Temperature By Country',
    geo = dict(
        showframe = True,
        showocean = True,
        oceancolor = 'rgb(0,255,255)',
        projection = dict(
        type = 'Mercator',
            
        ),
            ),
        )
fig = dict(data=data, layout=layout)
py.iplot(fig, validate=False, filename='worldmap2010')

Let’s look at the chart of annual temperature changes in certain continents (we take into consideration one country per continent and mark Greenland as the coldest place on Earth).

In [ ]:
continents = ['Africa', 'Europe', 'North America', 'South America', 'Australia', 'Asia']
continents_temps = country_temperatures[country_temperatures.Country.isin(continents)]
continents_temps['dt'] = pd.to_datetime(continents_temps.dt)
continents_temps['year'] = continents_temps['dt'].map(lambda x:x.year)
continents_temps['month'] = continents_temps['dt'].map(lambda x:x.month)

continents_temps = continents_temps[continents_temps.year > 1900].groupby(['Country', 'year'])['AverageTemperature'].mean().reset_index().sort_values(by=['Country', 'year'])


In [ ]:
traces = []
#colors = ['rgb(0, 255, 255)', 'rgb(255, 0, 255)', 'rgb(0, 0, 0)',
#          'rgb(255, 0, 0)', 'rgb(0, 255, 0)', 'rgb(0, 0, 255)']
colors = ['red', 'blue', 'black', 'green', 'pink', 'magenta']
years = continents_temps.year.unique()
for i, continent in enumerate(continents):
    traces.append(go.Scatter(
        x=years,
        y=continents_temps[continents_temps.Country == continent].AverageTemperature.values,
        mode='lines',
        name=continent,
        line=dict(color=colors[i]),
    ))
    
    
layout = go.Layout(
    xaxis=dict(title='Year'),
    yaxis=dict(title='Average Temperature, °C'),
    title='Average Land Temperature on the Continents',)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig)

## Temperature Difference By Country

In [ ]:
max_temps = country_temp[country_temp.year > 1900].groupby('Country')['AverageTemperature'].max()
min_temps = country_temp[country_temp.year > 1900].groupby('Country')['AverageTemperature'].min()
differences = (max_temps - min_temps).sort_values(ascending=False)

plt.figure(figsize=(15,8))
sns.barplot(y=differences.values[:50], x=differences.index[:50], palette='RdYlGn').set_title('Countries with Highest Difference between Max And Min Temperature')
plt.xticks(rotation=90);


In [ ]:
# And let's have a look at tge world map with temperature differences
data = [ dict(
        type = 'choropleth',
        autocolorscale = False,
        colorscale = 'Viridis',
        reversescale = True,
        showscale = True,
        locations = differences.index,
        z = differences.values,
        locationmode = 'country names',
        text = differences.index,
        marker = dict(
            line = dict(color = 'rgb(200,200,200)', width = 0.5)),
            colorbar = dict(autotick = True, tickprefix = '', 
            title = 'Temperature Difference')
            )
       ]

layout = dict(
    title = 'Temperature Difference By Country',
    geo = dict(
        showframe = True,
        showocean = True,
        oceancolor = 'rgb(0,255,255)',
        projection = dict(
        type = 'Mercator',
            
        ),
            ),
        )
fig = dict(data=data, layout=layout)
py.iplot(fig, validate=False, filename='worldmap2010')

# What about   South Africa ?

In [ ]:
#SA = city_temperatures[city_temperatures.Country == 'South Africa']
# Consider name changes
SA['City'].replace({'Pietersburg':'Polokwane'}, inplace=True)
#SA['month'] = SA.month.map(dict_month)
SA.head()

In [ ]:
sa_avg_temps = SA.groupby(['City'])['AverageTemperature'].mean().reset_index().sort_values(by='AverageTemperature',ascending=False)
#sa_avg_temps
plt.subplots(figsize=(8,30))
sns.barplot(y='City',  x='AverageTemperature', data=sa_avg_temps, palette='RdYlGn').set_title('Hottest Cities in SA')
plt.xlabel('Average Temperature');

In [ ]:
major_cities=SA[SA['City'].isin(['Johannesburg','Cape Town','Bloemfontein','Rustenburg', 'Nelspruit', 'Durban','Port Elizabeth','Pretoria','Polokwane','East London'])]

heatmap=major_cities.groupby(['City','month'])['AverageTemperature'].mean().reset_index().sort_values(by='month')
heatmap


In [ ]:
trace = go.Heatmap(z=heatmap['AverageTemperature'],
                   x=heatmap['month'].map(dict_month),
                   y=heatmap['City'],
                  colorscale='Viridis')
data=[trace]
layout = go.Layout(
    title='Average Temperature Of Major SA Cities By Month',
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='labelled-heatmap')

# Sequence Data



A sequence is any type of data that is in a particular order in which one thing follows another. 
Sequence data comes in many forms. Audio is a natural sequence. You can chop up an audio spectrogram into chunks and feed that into a machine learning model. Text is another form of sequences. You can break Text up into a sequence of characters or a sequence of words.

The simplest machine learning problem involving a sequence is a one to one problem.
<br>
![One2one](one2one.png "One 2 One")



In this case, we have one data input or tensor to the model and the model generates a prediction with the given input.

We can extend this formulation to allow for the model to make use of the past values of the input and the output. Now the output of the model is now fed back to the model as a new input. The model now can generate a new output and we can continue like this indefinitely.

![Many to One](many2one.png "Many 2 One")

There are called Recurrent Neural Networks (RNNs).
RNNs can retain state from one iteration to the next by using their own output as input for the next step.


A Time Series is a sequence where the data is in order, with a fixed time-difference between occurrence of successive data points. 
A Recurrent Neural Netowrk is a particular type of neural mnetwork that is  good at processing sequence data for predictions. That's because they implement a concept called sequential memory.
Sequential memory is a mechanism that makes it easier for the brain to recognize sequence patterns.


Traditional neural networks have an input layer, one or more hidden layers, and an output layer.

![Traditional Feed-Forward Neural Network](ffnn.png "Traditional Feed-Forward Neural Network")

Now we want to get a feed-forward neural network to be able to use previous information to effect later results. So we add a loop in the neural network that can pass prior information to later stages.

![RNN](rnn.gif "RNN")

Recurrent Neural Networks suffer from short-term memory. If a sequence is long enough, they’ll have a hard time carrying information from earlier time steps to later ones. So for us it will be diifcult to predict temperatures in 2020 while remembering what happened in the previous years.

To solve this problem Long Short Term Memory Networks (LSTM) were introduced. They have internal mechanisms called gates that can regulate the flow of information. These gates can learn which data in a sequence is important to keep or throw away. By doing that, it can pass relevant information down the long chain of sequences to make predictions. LSTM are capable or memorizing and carrying a "state". You can think of it as the “memory” of the network. So even information from the earlier time steps can make it’s way to later time steps, reducing the effects of short-term memory. As the cell state goes on its journey, information get’s added or removed to the cell state via gates.
![LSTM](lstm.png "LSTM")


If you want to know more, you can have a look at the followig articles: 

[Illustrated Guide to recurrent Neural Networks by Michael Nguyen](https://towardsdatascience.com/illustrated-guide-to-recurrent-neural-networks-79e5eb8049c9)

[Illustrated Guide to LSTM’s and GRU’s: A step by step explanation by Michael Nguyen](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21)

[A Guide For Time Series Prediction Using Recurrent Neural Networks (LSTMs) by Neelabh Pant
](https://blog.statsbot.co/time-series-prediction-using-recurrent-neural-networks-lstms-807fa6ca7f)

[Using LSTMs to forecast time-series by Ravindra Kompella
](https://towardsdatascience.com/using-lstms-to-forecast-time-series-4ab688386b1f)

In [ ]:
#global_mean.LandAverageTemperature.values

### Divide the dataset into training and validation sets
It is very important when you do time series to split train and test with respect to a certain date. So, you don’t want your test data to come before your training data.

Setting training set from: January 1901 to December 2000

Setting validation set from: January 2001 to December 2015

In [ ]:
ts = global_temperatures[['dt','LandAverageTemperature']]
rng = pd.date_range(start='1901-01-01', end='2015-12-01', freq = 'MS')

data = ts[ts.dt.dt.year>1900].LandAverageTemperature.values
series = pd.Series(data=data, index=rng)

In [ ]:
dataset = pd.DataFrame(data=data, index=rng)

train, test = dataset.iloc[:-180], dataset.iloc[-180:]
train.shape, test.shape

The next thing to do is normalize the dataset. You only need to fit and transform your training data and just transform your test data. The reason you do that is you don’t want to assume that you know the scale of your test data

In [ ]:
# Scaling values
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

sc = StandardScaler()
sc.fit(train)

scaled_train = sc.transform(train)
scaled_test = sc.transform(test)

### Linear Regression (One to One)

This is a simple regression model that will take one input and will spit out one output. This basically takes the temperature from the previous month and forecasts the temperature of the next month.

In [ ]:
# Linear Regression
from sklearn.linear_model import LinearRegression

X_train = scaled_train[:-1]
y_train = scaled_train[1:]

X_test = scaled_test[:-1]
#predict the test set results: insert last temperature from training set as first one of test set
X_test = np.insert(X_test, 0, X_train[-1][0]).reshape(-1,1)
#y_test = scaled_test[1:]
y_test = scaled_test


regressor = LinearRegression()
regressor.fit(X_train, y_train)



In [ ]:
# Predict test temperatures
y_pred = regressor.predict(X_test)


In [ ]:
# Metrics 
y_pred_inv_scaled = sc.inverse_transform(y_pred)
actuals = sc.inverse_transform(y_test)

print('MSE: %f'%mean_squared_error(actuals, y_pred_inv_scaled))
print('RMSE: %f'%np.sqrt(mean_squared_error(actuals, y_pred_inv_scaled)))
print('R-Squared: %f'%r2_score(actuals, y_pred_inv_scaled))


So the metric we need to memorize to compare with other models is: `RMSE: 2.099682`

In [ ]:
# Plot predictions vs actual values
plt.figure(figsize=(20,6))
plt.plot(actuals, label="Actuals temps")
plt.plot(y_pred_inv_scaled, label="Predicted") 
plt.legend(loc='upper right')
plt.xticks(np.arange(len(y_pred))[::6], pd.date_range(start='2001-01-01', end='2015-12-01', freq = '6MS').strftime(' %b %Y'), rotation="vertical")

plt.show()

So far it looks that the model fits properly to the training dta and performs well on tha validation data too. But what about generalizing to dates in the future?

In [ ]:
ext_y_pred = y_pred.copy()
last = ext_y_pred[-1].reshape(-1,1) # This is the value predicted for 2016
for i in range(24):
    next_month = regressor.predict(last)
    ext_y_pred=np.append(ext_y_pred, next_month)

y_pred_inv_scaled = sc.inverse_transform(ext_y_pred)
actuals = sc.inverse_transform(y_test)


plt.figure(figsize=(20,6))
plt.plot(actuals, label="Actuals temps")
plt.plot(y_pred_inv_scaled, label="Predicted")  
plt.legend(loc='upper right')
plt.xticks(np.arange(len(y_pred_inv_scaled))[::6], pd.date_range(start='2001-01-01', periods=len(y_pred_inv_scaled), 
                                                           freq = '6MS').strftime(' %b %Y'), rotation="vertical")
plt.show()

What does it mean?

The model is not good. It essentially is repeating the previous values and there is a slight shift. The model can't capture the seasonality of the data and therefore is not able to predict the future from the single previous value. 

# Many to One Models

In order to solve the problem above where the linear one-to-one model can't predict seasonality we will move now to a many-to-one model: we use a moving forward window of size 60 (5 years), which means we will use the first 60 data points as out input X to predict y1 — the 61st data point. Next, we will use the window between 1 to 61 data points as input X to predict y2 i.e., the 62nd data point and so on

In [ ]:
# Starting from training and test sequences
scaled_train.shape, scaled_test.shape

In [ ]:
# We build a dataframe with 61 columns where the 61st column is the label we want to predict
window_size = 60
df = pd.DataFrame(np.concatenate((scaled_train, scaled_test)))
df_s = df.copy()
for i in range(window_size):
    df = pd.concat([df, df_s.shift(-(i+1))], axis = 1)
    
df.dropna(axis=0, inplace=True)

In [ ]:
# First 60 columns are the features, 61st column is the label
df.shape

Now we can split our new dataframe into training and test

In [ ]:
# We will be left with 1140 data points to train our models and 180 to validate them
train, test = df.iloc[:-180, :], df.iloc[-180:,:]
train.shape, test.shape

In [ ]:
# Separating features from labels
X_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]
X_test = test.iloc[:,:-1]
y_test = test.iloc[:,-1]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
# We try first with a simple Linear Regression model
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Predicting on all the test data at once
y_pred = regressor.predict(X_test)

In [ ]:
# Retrieving temperatures data from the scaled ones

y_pred_inv_scaled = sc.inverse_transform(y_pred)
actuals = sc.inverse_transform(y_test)
actuals.shape

In [ ]:
print('MSE: %f'%mean_squared_error(actuals, y_pred_inv_scaled))
print('RMSE: %f'%np.sqrt(mean_squared_error(actuals, y_pred_inv_scaled)))
print('R-Squared: %f'%r2_score(actuals, y_pred_inv_scaled))

So we can already see that this model has a much better performance compared to the one-to-one linear regression 
whose RMSE was much higher at `2.099682`.

In [ ]:
# Visualize Predictions
plt.figure(figsize=(20,6))
plt.ylabel("Avg Temperature")
plt.plot(actuals, label="Actuals temps")
plt.plot(y_pred_inv_scaled, label="Predicted")
plt.legend(loc='upper right')
plt.xticks(np.arange(180)[::6], pd.date_range(start='2001-01-01', end='2015-12-01', freq = '6MS').strftime(' %b %Y'), rotation="vertical")
plt.show()

From the graph above it looks that the model fits very well the training and validation data, but what about generalizing to new data? we want to predict now temperatures for the next two years: 2016 and 2017


In [ ]:
ext_y_pred = y_pred.copy()
last = ext_y_pred[-60:] # This the temperatures predicted in the last 5 years

for i in range(24):
    last = last[None, :]
    next_month = regressor.predict(last)
    ext_y_pred = np.append(ext_y_pred, next_month)
    last = ext_y_pred[-60:]


y_pred_inv_scaled = sc.inverse_transform(ext_y_pred)
actuals = sc.inverse_transform(y_test)       

plt.figure(figsize=(20,6))
plt.ylabel("Avg Temperature")
plt.plot(actuals, label="Actuals temps")
plt.plot(y_pred_inv_scaled, label="Predicted")
plt.legend(loc='upper right')
plt.xticks(np.arange(len(y_pred_inv_scaled))[::6], pd.date_range(start='2001-01-01', periods=len(y_pred_inv_scaled), freq = '6MS').strftime('%b %Y'), rotation="vertical")
plt.show()

So it seems that the one-to-many linear regressor can capture the seasonality of the data too.

In [ ]:
# What's the average temperature predicted for June 2017? Hint index 180 = Jan 2016

y_pred_inv_scaled[180+5]

## Using FF Neural Netowrk

We will now try to predict temperatures using a Feed-Forward Neural Network. This network will of course have 60 input nodes, some hidden layers with relu activation functions, and one output node for the predicted temperature.
FF Neural Networks can virtually solve any problem thanks to the [Universal Approximation Theorem](https://en.wikipedia.org/wiki/Universal_approximation_theorem), that states that a feed-forward network with a single hidden layer containing a finite number of neurons can approximate continuous functions. You can find a more detailed explanation in this [blog post](https://towardsdatascience.com/can-neural-networks-really-learn-any-function-65e106617fc6).


In [ ]:
from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential
from keras import optimizers
from keras.optimizers import Adam

NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
adam = optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False)
NN_model.compile(loss='mse', optimizer=adam)
NN_model.summary()

In [ ]:
#from keras.callbacks.callbacks import ModelCheckpoint
#checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
#checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
#callbacks_list = [checkpoint]
#NN_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=callbacks_list)
NN_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
# We can visualize the Validation loss 

plt.plot(NN_model.history.history['val_loss'])
plt.xlabel("Epochs")
plt.ylabel("MSE")
plt.show()

In [ ]:
# Predictions
y_pred = NN_model.predict(X_test)
y_pred_inv_scaled = sc.inverse_transform(y_pred)
actuals = sc.inverse_transform(y_test)
actuals.shape

In [ ]:
print('MSE: %f'%mean_squared_error(actuals, y_pred_inv_scaled))
print('RMSE: %f'%np.sqrt(mean_squared_error(actuals, y_pred_inv_scaled)))
print('R-Squared: %f'%r2_score(actuals, y_pred_inv_scaled))

So we can see that this model performs a bit worse than the Linear Regression above. We will now try with LSTMs.

# LSTM

Long Short-Term Memory (LSTM) networks are a type of recurrent neural network capable of learning order dependence in sequence prediction problems.

Preparing the 3D input vector for the LSTM. 

Remember, the input vector for LSTM is 3D array: (num_samples, num_time_steps, num_features). In this case we have num of time steps = 50 and num_features = 1 (Extending the same analogy we saw in the previous point, that I found very useful in understanding why the input shape has to be like this — lets say, we have 50 words in one sentence and each word is represented by a word vector. So we need 50 time steps to go through each word vector in the sentence as an input to the LSTM at each time step. There is one sentence per observation and hence num_features = 1. 

In [ ]:
#Prepring rank 3 input tensors

train_X = train.iloc[:,:-1]
train_y = train.iloc[:,-1]
test_X = test.iloc[:,:-1]
test_y = test.iloc[:,-1]
train_X = train_X.values
train_y = train_y.values
test_X = test_X.values
test_y = test_y.values

train_X = train_X.reshape(train_X.shape[0],train_X.shape[1],1)
test_X = test_X.reshape(test_X.shape[0],test_X.shape[1],1)

train_X.shape, train_y.shape, test_X.shape, test_y.shape

In [ ]:
train_X.shape[1], train_X.shape[2]

In [ ]:
# Alternative LSTM
from keras.layers.recurrent import LSTM


lstm_model = Sequential()
lstm_model.add(LSTM(input_shape = (train_X.shape[1], train_X.shape[2]), output_dim=train_X.shape[1], return_sequences = True))
lstm_model.add(Dropout(0.5))
lstm_model.add(LSTM(256))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1))
lstm_model.add(Activation("linear"))
lstm_model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))
lstm_model.summary()


In [ ]:
import time
start = time.time()
lstm_model.fit(train_X,train_y, batch_size=32, nb_epoch=18, shuffle=False, validation_data=(test_X, test_y))

print("> Compilation Time : ", time.time() - start)

In [ ]:
plt.plot(lstm_model.history.history['val_loss'])
plt.xlabel("Epochs")
plt.ylabel("MSE")
plt.show()

In [ ]:
test_X.shape

In [ ]:
# Doing a prediction on all the test data at once
y_pred = lstm_model.predict(test_X)
y_pred_inv_scaled = sc.inverse_transform(y_pred)
actuals = sc.inverse_transform(test_y)
actuals.shape

In [ ]:
print('MSE: %f'%mean_squared_error(actuals, y_pred_inv_scaled))
print('RMSE: %f'%np.sqrt(mean_squared_error(actuals, y_pred_inv_scaled)))
print('R-Squared: %f'%r2_score(actuals, y_pred_inv_scaled))

The performances are similar to the FF Neural Net, we are confident that with some hyperparameter tuning we can get similar MSE to the Linear Regression model.

In [ ]:
ext_y_pred = y_pred.copy()
last = ext_y_pred[-60:].reshape(-1,1) # This is the value predicted for 2016

for i in range(24):
    last = last[None, :] 
    next_month = lstm_model.predict(last)
    ext_y_pred=np.append(ext_y_pred, next_month)
    last = ext_y_pred[-60:].reshape(-1,1)

y_pred_inv_scaled = sc.inverse_transform(ext_y_pred)
actuals = sc.inverse_transform(y_test)       

plt.figure(figsize=(20,6))
plt.ylabel("Avg Temperature")
plt.plot(actuals, label="Actuals temps")
plt.plot(y_pred_inv_scaled, label="Predicted")
plt.legend(loc='upper right')
plt.xticks(np.arange(len(y_pred_inv_scaled))[::6], pd.date_range(start='2001-01-01', periods=len(y_pred_inv_scaled), freq = '6MS').strftime('%b %Y'), rotation="vertical")
plt.show()
    

This model predicts according to seasonality correctly.

In [ ]:
def moving_test_window_preds(model, n_future_preds):

    ''' 
        model - The trained model we are using to predict
        n_future_preds - Represents the number of future predictions we want to make
                         This coincides with the number of windows that we will move forward
                         on the test data
    '''
    preds_moving = []                                    # Use this to store the prediction made on each test window
    moving_test_window = [test_X[0,:].tolist()]          # Creating the first test window
    moving_test_window = np.array(moving_test_window)    # Making it an numpy array
    
    for i in range(n_future_preds):
        preds_one_step = model.predict(moving_test_window) # Note that this is already a scaled prediction so no need to rescale this
        preds_moving.append(preds_one_step[0,0]) # get the value from the numpy 2D array and append to predictions
        preds_one_step = preds_one_step.reshape(1,1,1) # Reshaping the prediction to 3D array for concatenation with moving test window
        moving_test_window = np.concatenate((moving_test_window[:,1:,:], preds_one_step), axis=1) # This is the new moving test window, where the first element from the window has been removed and the prediction  has been appended to the end
        
    preds_moving = sc.inverse_transform(preds_moving)
    
    return preds_moving

In [ ]:
preds_moving = moving_test_window_preds(lstm_model, 50)

In [ ]:
plt.plot(actuals)
plt.plot(preds_moving)
plt.show()

In [ ]:
preds_moving = moving_test_window_preds(lstm_model, 50)
plt.plot(actuals)
plt.plot(preds_moving)
plt.show()

In [ ]:
# Possible exerrcises:

In [ ]:
# Can you show climate change impact in SA vs (Denmark?)

In [ ]:
# Can you predict the temperature of July 2020 in Cape Town?

In [ ]:
# How does to relate to some factors that can be changed like CO2 emissions, food production, natural disasters.

# Climate Change and Natural Disasters

In [ ]:
# Natural disasters
# Natural diasaster data is taken from https://ourworldindata.org/natural-disasters (data published by EMDAT (2019):
# OFDA/CRED International Disaster Database, Université catholique de Louvain – Brussels – Belgium)
nat_disaster_df = pd.read_csv('data/number-of-natural-disaster-events.csv')

nat_disaster_df.head()

In [ ]:
# Remove the 'Code' column
nat_disaster_df.drop(['Code'], axis = 1, inplace = True)

# Check the different types of 'Entity' values
nat_disaster_df['Entity'].value_counts()

In [ ]:
# Rearrange the data according to the disaster categories.
nat_disaster_df = nat_disaster_df.pivot(index = 'Year', columns = 'Entity', values = 'Number of reported natural disasters (reported disasters)')
nat_disaster_df.head()

In [ ]:
# Remove the 'Impact' column
nat_disaster_df.drop(['Impact'], axis = 1, inplace = True)

# Handle missing values and rename columns
nat_disaster_df.fillna(value = 0, inplace = True)
nat_disaster_df.dtypes

In [ ]:
#Plot the different types of Natural disaster to see if they increased over time

fig, ax = plt.subplots(2, 3, figsize=(15,12))

plt.subplot(2, 3, 1)
plt.title('Global occurrences of Flood 1950-2018', fontsize = 10)
nat_disaster_df.drop(['All natural disasters'], axis = 1).loc[1950:, 'Flood'].plot.bar()
plt.ylabel('Occurrences', fontsize = 15)
custom_ticks = np.arange(0, 2020-1950, 5)
custom_labels = np.arange(1950, 2020, 5)
plt.xticks(custom_ticks, custom_labels)

plt.subplot(2, 3, 2)
plt.title('Global occurrences of Drought 1950-2018', fontsize = 10)
nat_disaster_df.drop(['All natural disasters'], axis = 1).loc[1950:, 'Drought'].plot.bar()
plt.xticks(custom_ticks, custom_labels)

plt.subplot(2, 3, 3)
plt.title('Global occurrences of Wildfire 1950-2018', fontsize = 10)
nat_disaster_df.drop(['All natural disasters'], axis = 1).loc[1950:, 'Wildfire'].plot.bar()
plt.xticks(custom_ticks, custom_labels)

plt.subplot(2, 3, 4)
plt.title('Global occurrences of Earthquake 1950-2018', fontsize = 10)
nat_disaster_df.drop(['All natural disasters'], axis = 1).loc[1950:, 'Earthquake'].plot.bar()
plt.xticks(custom_ticks, custom_labels)


plt.subplot(2, 3, 5)
plt.title('Global occurrences of Extreme temperature 1950-2018', fontsize = 10)
nat_disaster_df.drop(['All natural disasters'], axis = 1).loc[1950:, 'Extreme temperature'].plot.bar()
plt.xticks(custom_ticks, custom_labels)

plt.subplot(2, 3, 6)
plt.title('Global occurrences of Extreme weather 1950-2018', fontsize = 10)
nat_disaster_df.drop(['All natural disasters'], axis = 1).loc[1950:, 'Extreme weather'].plot.bar()
plt.xticks(custom_ticks, custom_labels)

plt.show()

While some of these plots might seem inconclusive, the data on flood and extreme weather shows a marked increase of these phenomenon in the last years

Now we will plot natural disaster occurrences compared with the temperatures anomalies phenomenon we explored above

In [ ]:
# Plot all natural disasters occurrences and temperature anomaly for comparison
fig, ax = plt.subplots(figsize = (14, 8))
ax2 = ax.twinx()

#line1 = ax.plot(nat_disaster_df.loc[:2015, 'All natural disasters'], '-ro', markersize = 4, label = 'Natural disasters')
#line2 = ax2.plot(global_mean[global_mean.year>=1900].set_index('year').TemperatureAnomaly, 'b-', label = 'Temperature Anomaly')

line1 = ax.plot(nat_disaster_df.loc[:2015, 'All natural disasters'], '-r', label = 'Natural Disasters')
line2 = ax2.plot(global_mean[global_mean.year>=1900].set_index('year').TemperatureAnomaly, label = 'Temperature Anomaly')



lines = line1 + line2
labels = [l.get_label() for l in lines]

plt.title('Natural disasters  and Temperature Anomaly 1900-2015', fontsize = 15)
ax.set_xlabel('Year', fontsize = 15)
ax.set_ylabel('Disaster Occurrences', fontsize = 15)
ax2.set_ylabel('Temperature anomaly (Celsius)', fontsize = 15)
ax.legend(lines, labels, loc = 0, prop = {'size': 12})

plt.show()

Clearly shows a correlation between the increase in temperatures and natural disasters.

In [ ]:
# Load the economic damage dataset and store it in a dataframe
econ_dmg_df = pd.read_csv('data/economic-damage-from-natural-disasters.csv')


econ_dmg_df.head()

In [ ]:
# Possible questions:

# Can you show a correlation between Climate change and economic damage from natural disasters?
# Can you find correlations between the different types of natural disasters? 